In [1]:
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline  

import torch
import torch.nn as nn
import torch.optim as optim 

import itertools

from copy import deepcopy

import optimization_lib as optim_lib
import rao_blackwellization_lib as rb_lib
import baselines_lib as bs_lib 

from toy_experiment_lib import ToyExperiment

In [2]:
np.random.seed(454)
_ = torch.manual_seed(454)

# Fixed parameters

In [3]:
k = 5
p0 = torch.rand(k)

In [4]:
eta = torch.Tensor([5.])
toy_experiment = ToyExperiment(eta, p0)

# Get true gradient 

In [5]:
toy_experiment.set_parameter(eta)
full_loss = toy_experiment.get_full_loss()

In [6]:
full_loss.backward()

In [7]:
true_grad = toy_experiment.eta.grad

In [8]:
true_grad

tensor([ 2.3195])

# Integrate out REINFORCE analytically, check gradient 

In [9]:
toy_experiment.set_parameter(eta)
pm_loss = toy_experiment.get_pm_loss(topk = k, grad_estimator = bs_lib.reinforce)

TypeError: reinforce() got an unexpected keyword argument 'foo'

In [ ]:
pm_loss.backward()

In [ ]:
toy_experiment.eta.grad

In [ ]:
# why isn't this zero?
np.abs(true_grad - toy_experiment.eta.grad)

In [ ]:
def assert_close(x, y, tol): 
    diff = np.abs(x - y)
    assert diff < tol, 'difference = {}, tolerance = {}'.format(diff, tol)

In [ ]:
assert_close(true_grad, toy_experiment.eta.grad, tol = 1e-6)

# Sample and check REINFORCE 

In [ ]:
n_samples = 10000

reinforce_grads = torch.zeros(n_samples)

for i in range(n_samples): 
    toy_experiment.set_parameter(eta)
    pm_loss = toy_experiment.get_pm_loss(topk = 0, grad_estimator = bs_lib.reinforce)
    pm_loss.backward()
    
    reinforce_grads[i] = toy_experiment.eta.grad

In [ ]:
torch.mean(reinforce_grads)

In [ ]:
torch.std(reinforce_grads) / np.sqrt(n_samples)

In [ ]:
assert_close(torch.mean(reinforce_grads), 
             true_grad, 
             3 * torch.std(reinforce_grads) / np.sqrt(n_samples))

# Check REINFORCE with simple baseline 

In [ ]:
n_samples = 10000

reinforce_cv_grads = torch.zeros(n_samples)

for i in range(n_samples): 
    toy_experiment.set_parameter(eta)
    pm_loss = toy_experiment.get_pm_loss(topk = 0, grad_estimator = bs_lib.reinforce_w_double_sample_baseline)
    pm_loss.backward()
    
    reinforce_cv_grads[i] = toy_experiment.eta.grad

In [ ]:
torch.mean(reinforce_cv_grads)

In [ ]:
torch.std(reinforce_cv_grads) / np.sqrt(n_samples)

In [ ]:
assert_close(torch.mean(reinforce_cv_grads), 
             true_grad, 
             3 * torch.std(reinforce_cv_grads) / np.sqrt(n_samples))

# Check Rao-blackwellization of simple baseline

In [ ]:
n_samples = 10000

reinforce_rb_grads = torch.zeros(n_samples)

for i in range(n_samples): 
    toy_experiment.set_parameter(eta)
    pm_loss = toy_experiment.get_pm_loss(topk = 3, grad_estimator = bs_lib.reinforce_w_double_sample_baseline)
    pm_loss.backward()
    
    reinforce_rb_grads[i] = toy_experiment.eta.grad

In [ ]:
torch.mean(reinforce_rb_grads)

In [ ]:
torch.std(reinforce_rb_grads) / np.sqrt(n_samples)

In [ ]:
assert_close(torch.mean(reinforce_rb_grads), 
             true_grad, 
             3 * torch.std(reinforce_rb_grads) / np.sqrt(n_samples))